In [9]:
!pip install kiwipiepy


     ---------------------------------------- 1.1/1.1 MB 11.3 MB/s eta 0:00:00
     --------------------------------------- 30.8/30.8 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=c87a9241b1a2971b46d87467392aa34e8a5ebc17f9282c07ab9165e27005f47d
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\b6\6b\2a\b70c38baf0a75d295b0039fbf711850a22fe913b07d09d90c0
Successfully built kiwipiepy-model


In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Mecab
import matplotlib.pyplot as plt
from tqdm import tqdm
import pprint
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models import CoherenceModel
from gensim.models.callbacks import PerplexityMetric
import openpyxl

## 전처리

In [10]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [22]:
def extract_nouns(text):
    for token in kiwi.tokenize(text):
        if token.tag in {'NNG', 'NNP'}:
            yield token.form

In [23]:
path = r'C:\Users\user\OneDrive - KookminUNIV\바탕 화면\토픽모델링\빅카인즈_크롤링.xlsx'  

In [26]:
import pandas as pd
df = pd.read_excel(path, index_col=0)
df.head()

,Title,Content
Date,,
2023-11-27 조덕진,[사설] 생활권 침해 없는데 봉안당 불허···님비에 법원 제동,극단적인 지역이기주의에 지역 현안들이 발목이 잡혀있는 가운데 법원이 과도한 님비현상...
2023-11-27,[전남일보]기고·김정화>지방소멸 대책 신안에 있다,김정화 신안군농업기술센터소장\n김정화 소장\n“바보야! 문제는 경제야!”(“It’s...
2023-11-27 곽선정,“학생 수 감소했다면 학교용지비용 부담 필요 없어”,[KBS 광주]\n취학 인구가 감소했는데도 학교용지부담금을 부과한 행정처분은 부당하...
2023-11-27 2015004;2017021 기자,서삼석 예결위원장 “올해 전 부처가 증액 요구…특활비 무조건 삭감보다 투명성 제고”...,서삼석 국회 예산결산특별위원회 위원장이 17일 오후 서울 여의도 국회에서 헤럴드경제...
2023-11-27 김진영,"경단녀서 꽃가게 사장님으로… ""지방은 기회의 공간"" [지방시대 지방영웅]",<4> 전남 목포 '두리하나 플라워' 임효백씨 아이 때문에 결심한 '지방살이'서 꿈...


In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=extract_nouns, min_df=10)
dtm = cv.fit_transform(df.Content)

In [28]:
word_count = pd.DataFrame({
    'word': cv.get_feature_names_out(),
    'count': dtm.sum(axis=0).flat
})

word_count.sort_values('count', ascending=False).head(20)


,word,count
5229,지역,10899
639,교육,9195
2570,사업,8707
4751,전남,8222
5234,지원,7375
1855,목포,6445
3347,신안군,5113
1939,문화,5105
5602,추진,4215
2883,섬,3975


In [40]:
len(word_count)

6637

## LDA

In [29]:
from gensim.matutils import Sparse2Corpus
words = cv.get_feature_names_out().tolist()
corpus = Sparse2Corpus(dtm.T)
id2word = dict(enumerate(words))

In [30]:
#0번 문서의 단어 수
corpus[0]

[(812, 1),
 (5229, 8),
 (6406, 1),
 (33, 1),
 (2227, 2),
 (527, 2),
 (6402, 3),
 (4953, 2),
 (6376, 1),
 (5079, 1),
 (4898, 4),
 (6418, 2),
 (947, 3),
 (3265, 6),
 (4330, 2),
 (4567, 3),
 (1189, 2),
 (238, 2),
 (4589, 1),
 (5430, 2),
 (6364, 1),
 (4751, 1),
 (1855, 1),
 (3283, 1),
 (2675, 1),
 (3955, 4),
 (2243, 4),
 (6378, 1),
 (5688, 1),
 (2982, 1),
 (5969, 2),
 (5200, 1),
 (1866, 4),
 (4006, 1),
 (3149, 1),
 (231, 2),
 (1887, 1),
 (560, 1),
 (3359, 1),
 (3885, 1),
 (6310, 2),
 (5068, 2),
 (831, 1),
 (5083, 2),
 (4921, 1),
 (3647, 1),
 (639, 1),
 (6527, 1),
 (3471, 1),
 (549, 2),
 (2222, 1),
 (2588, 2),
 (2376, 1),
 (5695, 1),
 (2952, 1),
 (4951, 1),
 (1929, 3),
 (870, 1),
 (4222, 1),
 (255, 1),
 (5102, 1),
 (4703, 1),
 (2568, 1),
 (2020, 1),
 (2760, 1),
 (6192, 1),
 (6195, 1),
 (5721, 1),
 (4294, 1),
 (2604, 2),
 (885, 1),
 (4936, 1),
 (4323, 1),
 (4937, 3),
 (6354, 1),
 (951, 1),
 (2507, 1),
 (330, 1),
 (81, 1),
 (5024, 1),
 (2132, 1),
 (3681, 1),
 (2410, 1)]

In [31]:
from gensim.models.ldamodel import LdaModel
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=15,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    iterations=400,
    alpha='auto',
    eta='auto',
    per_word_topics=True)


In [32]:
lda_model.show_topic(0)

[('마을', 0.024189066),
 ('세계', 0.024169063),
 ('자원', 0.021981416),
 ('관광', 0.021456838),
 ('전시', 0.013096576),
 ('자연', 0.012688234),
 ('사람', 0.01132914),
 ('생태', 0.01130092),
 ('바다', 0.01099736),
 ('길', 0.010834198)]

In [34]:
word_idx = words.index('사람')
lda_model.get_term_topics(word_idx)


[(0, 0.011322198)]

In [35]:
doc_idx = 0
lda_model.get_document_topics(corpus[doc_idx])

[(0, 0.04763123),
 (2, 0.015201082),
 (3, 0.034791715),
 (4, 0.037005927),
 (6, 0.1157737),
 (7, 0.09732184),
 (8, 0.048003312),
 (9, 0.029302891),
 (10, 0.010108594),
 (14, 0.5548392)]

## 평가

In [36]:
lda_model.log_perplexity(corpus)
 #0에가까울수록 성능 높음

-9.92935250462413

In [37]:
topn = 25
top_words = set()

for topic in range(lda_model.num_topics):
    for word, prob in lda_model.show_topic(topic, topn=topn):
        top_words.add(word)

len(top_words) / (lda_model.num_topics * topn)

#1에가까울수록 성능 높음

0.8106666666666666

In [38]:
from gensim.corpora.dictionary import Dictionary
dic = Dictionary()
dic.id2token = id2word
dic.token2id = {w: i for i, w in id2word.items()}

In [39]:
import pyLDAvis.gensim
p = pyLDAvis.gensim.prepare(
    lda_model, corpus, dic, sort_topics=False)
pyLDAvis.display(p)

c:\Users\user\anaconda3\envs\PY39_\lib\site-packages\pandas\core\dtypes\cast.py:1429: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
